In [69]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
#from transformers import pipeline
# For fine-tuning
import torch
#from torch import nn
#from torch.utils.data import Dataset, DataLoader
# For processing
import numpy as np
import pandas as pd
import os
import json

In [70]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [71]:
folder_path = "./dreams"
count = 0
all_files = []
for i in os.listdir(folder_path):
    if i.endswith('.json'):
        count+=1
        all_files.append(i)
dreams = []
for i in all_files:
    full_name  = os.path.join(folder_path,i)
    with open(full_name,'r') as f:
        data = json.load(f)
        for x in data['dreams']:
            dreams.append({
                "content":x.get("content","")
            })
              
df = pd.DataFrame(dreams)
#print(count)
df.size

22415

In [72]:

from torch import device

model = BertModel.from_pretrained('bert-base-uncased')
#tokenizer will break the sentence down and split into word array and then 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "I had a dream about flying"
#this is the input that automozatically converts into nunmbers
def batch_get_embeddings(texts, batch_size=8):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(
            batch, 
            return_tensors='pt',
            padding=True, 
            truncation=True, 
            max_length=512
        )
        inputs =  {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            batch_emb = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Shape: (batch_size, hidden_dim)
            embeddings.extend(batch_emb)
    return embeddings

texts = df['content'].tolist()
embeddings = batch_get_embeddings(texts)
df['embedding'] = embeddings



TypeError: to() received an invalid combination of arguments - got (type), but expected one of:
 * (torch.device device = None, torch.dtype dtype = None, bool non_blocking = False, bool copy = False, *, torch.memory_format memory_format = None)
 * (torch.dtype dtype, bool non_blocking = False, bool copy = False, *, torch.memory_format memory_format = None)
 * (Tensor tensor, bool non_blocking = False, bool copy = False, *, torch.memory_format memory_format = None)


In [58]:
#analysis
import tensorflow as tf

# a good k for th etop k elements with the greatest magnitude
# Assuming pooled_output is your BERT pooled output
tf_po = tf.constant(pooled_output.numpy())
#we need to normalize the vector to fidn the magnitude
mean = tf.reduce_mean(tf_po)

mag_vec = tf_po - mean
#type(mean)
std_dev = tf.math.reduce_std(mag_vec)
normalized = mag_vec/std_dev
normalized

# 

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-1.5906411 , -0.43908513,  0.12600473,  0.9370224 ,  0.0497518 ,
        -0.2983595 ,  1.3857278 ,  0.36702126, -0.07756235, -1.8350252 ,
         0.06647883,  1.0917829 ,  1.791535  , -0.4422847 ,  1.7030795 ,
        -0.80148584, -0.1388493 , -1.0381039 ,  0.34101674, -0.7799106 ,
         1.0306836 ,  1.7576712 ,  0.8940628 ,  0.4766203 ,  0.7136126 ,
         1.2967691 , -0.9625652 ,  1.6729654 ,  1.7240939 ,  1.2781806 ,
        -1.0599849 ,  0.18868391, -1.8109761 , -0.2853855 ,  0.40937552,
        -1.8077661 ,  0.25897694, -1.3084244 ,  0.17061041,  0.2329538 ,
        -1.6804143 ,  0.4351838 ,  1.8170068 , -0.96891546, -0.04248921,
        -0.50476813, -1.8352612 ,  0.3248403 , -1.5627068 , -0.0150576 ,
         0.08029152, -0.16575864,  0.19295026,  0.60096115,  0.68903756,
         0.36531174, -0.39197847, -0.00652819, -0.348254  , -0.8608494 ,
        -1.1399164 ,  0.48556918, -0.2658665 , -1.5484265 ,  0.36107385,
 

In [59]:
#sort the vector by the top 50 elements
normalize = tf.abs(normalized)
result_top_50 = tf.math.top_k(
    normalize,
    k = 200,
)
#result_top_50


In [62]:
# Get top 50 indices
top_50_indices = result_top_50.indices.numpy()[0]

# Convert to tokens and filter out '[unused*]' tokens
top_50_tokens = [
    tokenizer.convert_ids_to_tokens(int(i)) 
    for i in top_50_indices 
    if not tokenizer.convert_ids_to_tokens(int(i)).startswith("[unused")
    and tokenizer.convert_ids_to_tokens(int(i))!="[PAD]"
]
print(top_50_tokens)

['[PAD]']
